# 03. GRU (Gated Recurrent Unit): LSTM의 다이어트 버전
### 과목: NLP Sequence Modeling
---

## 1. LSTM이 너무 무거워요!
LSTM은 성능이 좋지만, 계산할 게 너무 많습니다.
- 문(Gate)이 3개나 있고(망각, 입력, 출력), 기억 장소도 2개(Hidden, Cell)나 됩니다.
- 학습 시간이 오래 걸리고 메모리도 많이 먹습니다.

## 2. GRU: "필요 없는 거 빼고 합치자!"
2014년, 조경현 교수님 팀이 제안한 모델입니다. LSTM을 단순화하면서 성능은 유치했습니다.
- **다이어트 비법**:
    1.  **게이트 통합**: 망각 게이트 + 입력 게이트 -> **업데이트 게이트(Update Gate)** 하나로 합침.
    2.  **기억 통합**: Cell State를 없애고 **Hidden State** 하나로 퉁침.
    3.  **출력 게이트 삭제**: 그냥 Hidden State를 그대로 내보냄.

> **결론**: "LSTM보다 구조는 훨씬 간단한데 성능은 비슷하네? 완전 이득!"

---

## 3. 실습: PyTorch로 GRU 써보기
사용법은 `nn.LSTM`과 비슷하지만, **Cell State가 없다**는 점이 중요합니다. (RNN처럼 Hidden만 있음)

In [ ]:
import torch
import torch.nn as nn

# --- [1] 데이터 준비 ---
batch_size = 2
seq_len = 3
input_size = 4
hidden_size = 5

x = torch.randn(batch_size, seq_len, input_size)
print(f"입력 데이터: {x.shape}")

# --- [2] GRU 모델 생성 ---
# LSTM과 선언 방식은 똑같습니다.
gru = nn.GRU(input_size, hidden_size, batch_first=True)

# --- [3] 실행 (Forward) ---
# 차이점: 리턴값이 output과 hidden 2개뿐이다! (Cell State 없음)
output, hidden = gru(x)

print("\n--- 결과 확인 ---")
print(f"Output: {output.shape} (모든 시점의 기억)")
print(f"Hidden State: {hidden.shape} (최종 기억)")
print("Cell State: 없음! (심플하죠?)")

---
## 4. 실전: IMDB 감성 분석 (다이어트 버전)
코드는 LSTM과 99% 동일합니다. 모델만 바꿔 끼워봅시다.

In [ ]:
from tensorflow.keras.datasets import imdb
import torch.nn.functional as F

# 데이터 로드 및 전처리는 앞에서 했으므로 빠르게 넘어갑니다.
vocab_size = 300
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
X_train, y_train = X_train[:15000], y_train[:15000]
X_test, y_test = X_test[:10000], y_test[:10000]

seq_len = 100
def pad_sequences(sequences, max_len):
    padded_list = []
    for seq in sequences:
        seq_tensor = torch.tensor(seq, dtype=torch.long)
        if len(seq_tensor) < max_len:
            padded = F.pad(seq_tensor, (0, max_len - len(seq_tensor)), value=0)
        else:
            padded = seq_tensor[:max_len]
        padded_list.append(padded)
    return torch.stack(padded_list)

X_train_padded = pad_sequences(X_train, seq_len)
X_train_onehot = F.one_hot(X_train_padded, num_classes=vocab_size).float()

In [ ]:
import torch.nn as nn

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        # ★ GRU 사용 ★
        self.gru = nn.GRU(
            input_size=input_dim, 
            hidden_size=hidden_dim, 
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # GRU는 반환값이 2개 (output, hidden)
        output, hidden = self.gru(x)
        
        # 마지막 hidden만 사용
        last_hidden = hidden[-1]
        score = self.fc(last_hidden)
        return score

model = GRUNet(vocab_size, hidden_dim=16, output_dim=1)
print("GRU 모델 준비 완료! (LSTM보다 가벼움)")

In [ ]:
# 학습 코드 (동일)
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

y_train_tensor = torch.tensor(y_train).float().unsqueeze(1)
train_dataset = TensorDataset(X_train_onehot, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
print("GRU 학습 시작...")
for epoch in range(3):
    total_loss = 0
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

### 🌟 오늘의 결론
1. **GRU**는 LSTM의 동생이다. (구조가 훨씬 간단함)
2. 성능은 LSTM과 비슷하면서 학습 속도는 더 빠르다.
3. 데이터가 적을 땐 GRU가 유리하고, 데이터가 아주 많으면 LSTM이 유리할 수도 있다.